# Automatic natural language processing

### Autocompletion and sentence generation with n-gram language models

## The team

- Grover-Brando Tovar Oblitas
- Andy Chen
- Marco Mudenge

## Description:

This second notebook will be dedicated to the use of n-grams in natural language processing (NLP). N-grams are sequences of n consecutive words in text, and they are present in many applications of NLP.

During this lab, we will explore how n-grams are used for tasks such as word prediction, autocompletion, and text generation. Here are some concrete examples of their use:

Word Prediction: N-grams are used to predict the next word given the previous context. For example, in the sentence "I think therefore I ____", a bigram model might suggest "am" as the next word, based on previous observations.

Autocomplete: Search engines use n-grams to provide search suggestions when you start typing a phrase. They are based on the prefixes of the current word and on the sentences written by the user.

Text Generation: N-grams can be used to automatically generate text. They help a model predict next words based on the probabilities of words that can be generated depending on the context.

You will also explore a way to assess the quality of generative language models using perplexity. Perplexity is a metric that evaluates the confidence with which a model is able to predict a sequence of words in text. The lower the perplexity, the more accurately the model is able to predict. That is, perplexity is a measure of how “surprised” a model is when exposed to a new set of words. We will use this metric to evaluate our n-gram based models.

<a name='1'></a>
## 1. Loading and pre-processing data

<a name='1.1'></a>
### 1.1 Loading data

The data you will use in this work is contained in the file [trump.txt](./trump.txt)

Read the contents of this file and store it in a data variable.

Then display the first 300 characters

In [1]:

filename = "trump.txt"
with open(filename, encoding="utf8") as f:
    data = f.read()

print(data[:300])

Thank you very much.
We had an amazing convention.
That was one of the best.
I think it was one of the best ever.
In terms -- in terms of enthusiasm, in terms of I think what it represents, getting our word out.
Ivanka was incredible last night.
She did an incredible job.
And so many of the speakers


<a name='1.2'></a>
### 1.2 Segmentation

Pre-process the data using the following steps:

1. Remove capital letters.
2. Replace “\n” with spaces
3. Separate the data into sentences using the following delimiters `.`, `?` and `!` as a separator.
4. Remove punctuation marks (Be careful to keep spaces).
5. Remove empty sentences.
6. Segment sentences with the nltk.word_tokenize() function

Then use your function to preprocess the dataset.

In [2]:
import nltk
import re

def preprocess(data):

    # Remove capital letters
    data = data.lower()

    # Remove line breaks
    data = data.replace('\n', ' ')

    # Split into sentences using ., !, ? as delimiters
    data = re.split('[.!?]', data)

    # Remove punctuation
    data = [re.sub(r'[^\w\s]', '', s) for s in data]

    # Remove empty sentences
    data = [sentence for sentence in data if sentence != '']

    # Tokenize each sentence into words
    data = [nltk.word_tokenize(sentence) for sentence in data]

    return data

In [3]:
# test
x = "Cats are independent.\nDogs are faithful."
preprocess(x)

[['cats', 'are', 'independent'], ['dogs', 'are', 'faithful']]

<a name='1.3'></a>
### 1.3 Creating training and testing sets

**Randomly** sample 80% of the data for the training set. Keep 20% for the test set. Use sklearn's train_test_split function. Store results in variables.

In [4]:
from sklearn.model_selection import train_test_split

data = preprocess(data)

X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)

<a name='1.4'></a>
### 1.4 Vocabulary construction

As in TP1, build a vocabulary from the training data. You can take your code from TP1.

Complete the **build_voc** function which returns a list of tokens that are present at least n times (threshold passed as a parameter) in the list of examples (also passed as a parameter). You can use the collections.Counter class.

Then call this function to build your vocabulary from the training set **using threshold=2**. Print the vocabulary size.

In [5]:
from collections import Counter

def build_voc(documents, threshold = 0):

    tokens = [token for sentence in documents for token in sentence]

    counter = Counter(tokens)

    vocabulary = [token for token, count in counter .items() if count >= threshold]
    
    return vocabulary + ['<e>']


In [6]:
X_train_vocabulary = build_voc(X_train, 2)
print('Nombre de mots dans le vocabulaire (threshold de 2): {}'.format(len(X_train_vocabulary)))

Nombre de mots dans le vocabulaire (threshold de 2): 5609


<a name='1.5'></a>
### 1.5 Non-vocabulary words

If your model performs autocompletion, but it encounters a word that it has never seen during training, the model will not be able to predict the next word because there is no occurrence for the current word.

These words are called Out of Vocabulary <b>OOV</b> words.
The percentage of unknown words in the test set is called the <b>OOV</b> word rate.

To handle unknown words when predicting, use a special 'unk' token to represent all unknown words. More specifically, the technique you will use will be the following:

Complete the replace_oov function which converts all non-vocabulary words to the '\<unk\>' token.

Then call your function on your training and testing corpus.


In [7]:
import copy

def replace_oov(tokenized_sentences, voc, show_stats = True):

    new_tokenized_sentences = []
    sentences_copy = copy.deepcopy(tokenized_sentences) # Avoid modifying the original list

    word_replace_count = 0
    for sentence in sentences_copy:
        for i, token in enumerate(sentence):
            if token not in voc:
                sentence[i] = '<unk>'
                word_replace_count += 1
        new_tokenized_sentences.append(sentence)

    if show_stats:
        print('OOV words replaced: {}'.format(word_replace_count))
    
    return new_tokenized_sentences

In [8]:
tokenized_sentences = [["cats", "sleep"], ["mice", "eat"], ["cats", "and", "mice"]]
vocabulary = build_voc([["cats", "sleep"], ["mice", "eat"], ["cats", "and", "mice"]], 2)
tmp_replaced_tokenized_sentences = replace_oov(tokenized_sentences, vocabulary)
print(f"Phrase initiale:")
print(tokenized_sentences)
print(f"Phrase segmentée avec'<unk>':")
print(tmp_replaced_tokenized_sentences)

OOV words replaced: 3
Phrase initiale:
[['cats', 'sleep'], ['mice', 'eat'], ['cats', 'and', 'mice']]
Phrase segmentée avec'<unk>':
[['cats', '<unk>'], ['mice', '<unk>'], ['cats', '<unk>', 'mice']]


In [9]:
X_train = replace_oov(X_train, X_train_vocabulary)
X_test = replace_oov(X_test, X_train_vocabulary)

OOV words replaced: 2738
OOV words replaced: 1239


<a name='2'></a>
## 2. N-gram language patterns

In this section, you will develop an n-grams language model. We will use the formula:

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_t)}{C(w_{ t-1}\dots w_{t-n})} \tag{2} $$

- The $C(\cdots)$ function represents the number of occurrences of the given sequence.
- $\hat{P}$ means the estimate of $P$.

You can estimate this probability by counting the occurrences of these word sequences in the training data.

<a name='2.1'></a>
### 2.1 Frequency of n-grams

You will start by implementing a function that calculates the frequency of n-grams for an arbitrary number $n$.

You need to pre-process the sentence by adding $n$ start-of-sentence markers "\<s\>" to indicate the start of the sentence.

- For example, in a bigram model (N=2), the sequence should start with two starting tokens "\<s\>\<s\>". So, if the sentence is "I like food", change it to "\<s\>\<s\> I like food".
- Also add an ending token "\<e\>" so the model can predict when to end a sentence.
    

In this implementation, you need to store the occurrences of n-grams as a dictionary.

- The key of each key-value pair in the dictionary is a tuple of n words (not a list).
- The value in the key-value pair is the number of occurrences.

In [10]:

def count_n_grams(data, n, start_token='<s>', end_token = '<e>'):

    n_grams = {}

    for sentence in data:
        # Add start and end tokens
        sentence = [start_token] * n + sentence + [end_token]

        sentence = tuple(sentence)

        # Create n-grams
        n_grams_in_sentence = zip(*[sentence[i:] for i in range(n)])

        # Count n-grams
        for n_gram in n_grams_in_sentence:
            if n_gram in n_grams:
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1

    return n_grams

In [11]:
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
print("Unigrammes:")
print(count_n_grams(sentences, 1))
print("Bigrammes:")
print(count_n_grams(sentences, 2))

Unigrammes:
{('<s>',): 2, ('i',): 1, ('have',): 1, ('a',): 1, ('mouse',): 2, ('<e>',): 2, ('this',): 1, ('likes',): 1, ('cats',): 1}
Bigrammes:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'have'): 1, ('have', 'a'): 1, ('a', 'mouse'): 1, ('mouse', '<e>'): 1, ('<s>', 'this'): 1, ('this', 'mouse'): 1, ('mouse', 'likes'): 1, ('likes', 'cats'): 1, ('cats', '<e>'): 1}


<a name='2.2'></a>
### 2.2 MLE maximum likelihood estimate

#### 2.2.1 Calculation of probability for a word


Then estimate the probability of a word given the previous 'n' words with the frequencies obtained.

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_t)}{C(w_{ t-1}\dots w_{t-n})} \tag{2}$$


The function takes as input:

- word: the word whose probability we want to estimate
- previous_n_gram: the previous n-gram, in tuple form
- n_gram_counts: A dictionary where the key is the n-gram and the value is the frequency of that n-gram.
- n_plus1_gram_counts: Another dictionary, which you will use to find the frequency of the previous n-gram plus the current word.

In [12]:

def estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts):

    previous_n_gram = tuple(previous_n_gram)
    previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0

    n_plus1_gram = previous_n_gram + (word,)
    n_plus1_gram_count = n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts else 0

    probability = n_plus1_gram_count / previous_n_gram_count

    return probability

#### 2.2.1 What's wrong with this function? What pitfall could we encounter? Reply with an example.

> The probability of a sentence containing an N-gram never observed will always be 0.
For example, with the corpus [['i', 'have', 'a', 'mouse'],
              ['this', 'mouse', 'likes', 'cats']], if we wanted to estimate the probability of the word 'the', it will return 0

<a name='2.3'></a>
### 2.3 Add-k smoothing

You will now modify your previous function using add-k smoothing.

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_n) + k}{C( w_{t-1}\dots w_{t-n}) + k|V|} \tag{3} $$

Recode the function in number 2.2 by adding a smoothing constant $k$ and the vocabulary size as additional parameters.

In [13]:

def estimate_probability_smoothing(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    previous_n_gram = tuple(previous_n_gram)
    previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0

    denominator = previous_n_gram_count + k*vocabulary_size

    n_plus1_gram = previous_n_gram + (word,)
    n_plus1_gram_count = n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts else 0

    numerator = n_plus1_gram_count + k

    probability = numerator / denominator

    return probability

In [14]:
# test
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = list(set(sentences[0] + sentences[1]))

bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
tmp_prob = estimate_probability_smoothing("have", ['<s>', 'i'], bigram_counts, trigram_counts, len(unique_words), k=1)

print(f" La probabilité de 'have' étant donné le mot précédent 'i' est: {tmp_prob:.4f}")

 La probabilité de 'have' étant donné le mot précédent 'i' est: 0.2500


<a name='2.4'></a>
### 2.4 Calculation of probabilities of n-grams

#### 2.4.1. Estimation of probabilities
Complete the estimate_probabilities function which calculates for each word in the vocabulary the probability of being generated using the add-k smoothing function.

Don't forget to add the special token "\<e\>" to the vocabulary

This function takes as input:
- previous_n_gram: the previous n-gram, in tuple form
- n_gram_counts: A dictionary where the key is the n-gram and the value is the frequency of that n-gram.
- n_plus1_gram_counts: Another dictionary, which you will use to find the frequency of the previous n-gram plus the current word.
- vocabulary: vocabulary
- k: the smoothing constant

The function returns a dictionary having as keys all the words in the vocabulary as well as their probability of being generated

In [15]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):

    previous_n_gram = tuple(previous_n_gram)

    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability_smoothing(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, len(vocabulary), k=k)
        probabilities[word] = probability

    return probabilities

In [16]:
# test
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = build_voc(sentences, threshold=0)
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
estimate_probabilities(["a"], unigram_counts, bigram_counts, unique_words, k=1)

{'i': 0.1111111111111111,
 'have': 0.1111111111111111,
 'a': 0.1111111111111111,
 'mouse': 0.2222222222222222,
 'this': 0.1111111111111111,
 'likes': 0.1111111111111111,
 'cats': 0.1111111111111111,
 '<e>': 0.1111111111111111}

#### 2.4.2. Probabilities given a context

Now display the probabilities of tri-grams given the context "i will" using the training data. Show only the 10 most likely words in descending order of probability. Use K=1

In [17]:
X_train_unigrams = count_n_grams(X_train, 1)
X_train_bigrams = count_n_grams(X_train, 2)
X_train_trigrams = count_n_grams(X_train, 3)
X_train_quadgrams = count_n_grams(X_train, 4)
X_train_quintgrams = count_n_grams(X_train, 5)

In [18]:
print("Probabilités des tri-grammes étant donné le context 'i will':")
i_will_prob = estimate_probabilities(['i', 'will'], X_train_bigrams, X_train_trigrams, X_train_vocabulary, k=1)
i_will_prob_top = sorted(i_will_prob.items(), key=lambda x: x[1], reverse=True)
i_will_prob_top[:10]

Probabilités des tri-grammes étant donné le context 'i will':


[('tell', 0.008267251560654632),
 ('fight', 0.005567740846971487),
 ('fix', 0.005230302007761093),
 ('be', 0.004555424329340307),
 ('never', 0.004049266070524717),
 ('say', 0.0035431078117091276),
 ('not', 0.002193352454867555),
 ('ask', 0.0020246330352623586),
 ('also', 0.0016871941960519656),
 ('work', 0.0016871941960519656)]

<a name='3'></a>
## 3. Perplexity

In this section, you will generate the perplexity score to evaluate your model on the test set.

To calculate the perplexity score of a sentence on an n-gram model, use:

$$PP(W) =\sqrt[N]{ \prod_{t=1}^{N} \frac{1}{P(w_t | w_{t-n} \cdots w_{t-1})} } \ tag{4.1}$$

where N = the number of tokens in the sentence including the token \<e\>
and P = the probability of generating the token $w_t$

The higher the probabilities, the lower the perplexity will be.

<a name='3.1'></a>
### 3.1. Calculation of perplexity
Complete the `calculate_perplexity` function, which for a given sentence gives us the perplexity score. This function takes as input:


- sentence: The sentence for which you must calculate the perplexity
- n_gram_counts: A dictionary where the key is the n-gram and the value is the frequency of that n-gram.
- n_plus1_gram_counts: Another dictionary, which you will use to find the frequency of the previous n-gram plus the current word.
- vocabulary_size: the size of the vocabulary
- k: the smoothing constant

In [19]:

def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):

    # Calculate perplexity for one sentence
    n = len(list(n_gram_counts.keys())[0]) # n is the n of the n-gram we use to estimate the probabilities

    sentence = ["<s>"] * n + sentence + ["<e>"]
    sentence = tuple(sentence)

    N = len(sentence) - n

    product_pi = 1.0

    for t in range(n, N+n):
        word = sentence[t]
        previous_n_gram = sentence[t-n:t]
        probability = estimate_probability_smoothing(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=k)
        product_pi *= 1/probability

    perplexity = product_pi**(1.0/float(N))

    return perplexity


In [20]:
# test

sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)


perplexity = calculate_perplexity(sentences[0],
                                         unigram_counts, bigram_counts,
                                         len(unique_words), k=1.0)
print(f"Perplexité de la première phrase: {perplexity:.4f}")


Perplexité de la première phrase: 4.1930


<a name='3.2'></a>
### 3.2. Perplexity on a training sentence
Calculate and display the perplexity of bi-gram, tri-gram and quad-gram models using your `calculate_perplexity` function defined above on the first sentence of your training corpus. Use K=0.01 here.

In [21]:
perplexity_bigram = calculate_perplexity(X_train[0], 
                                         X_train_unigrams, X_train_bigrams, 
                                         len(X_train_vocabulary), k=0.01)

perplexity_trigram = calculate_perplexity(X_train[0],
                                          X_train_bigrams, X_train_trigrams,
                                          len(X_train_vocabulary), k=0.01)

perplexity_quadrigram = calculate_perplexity(X_train[0],
                                             X_train_trigrams, X_train_quadgrams,
                                             len(X_train_vocabulary), k=0.01)

print(f"Perplexité de la première phrase (bi-gramme): {perplexity_bigram:.4f}")
print(f"Perplexité de la première phrase (tri-gramme): {perplexity_trigram:.4f}")
print(f"Perplexité de la première phrase (quadri-gramme): {perplexity_quadrigram:.4f}")

Perplexité de la première phrase (bi-gramme): 49.2881
Perplexité de la première phrase (tri-gramme): 29.3860
Perplexité de la première phrase (quadri-gramme): 34.5169


<a name='3.3'></a>
### 3.3. Perplexity of the test corpus

#### 3.3.1. You can now calculate and display the perplexity of bi-gram, tri-gram and quad-gram models on your test corpus. K=1 here. (4 points)

To calculate the perplexity of a corpus of *m* sentences, simply follow the following formula:

Let $N$ be the total number of tokens in the test corpus C and $N_i$ be the number of tokens in sentence i.

$$Perplexity(C) = \Big(\frac{1}{P(s_1, ..., s_m)}\Big)^{1/N}$$
$$P(s_1, ..., s_m) = \prod_{i=1}^{m} p(s_i)$$
$$p(s_i) = \prod_{t=1}^{N_i} \hat{P}(w_t | w_{t-n} \cdots w_{t-1})$$

Since it is a multiplication of probabilities (located between 0 and 1), the product becomes zero very quickly. This is why it is more efficient to transform to a logarithmic space to transform multiplications into addition. This gives the following formula:

$$LogPerplexity(C) = 2^{-\frac{1}{N} \sum_{k=1}^{m} log_{2} \; p(s_k)}$$

In [22]:
import math

def calculate_perplexity_corpus(corpus, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):

    N = sum([len(sentence) + 1 for sentence in corpus])

    n = len(list(n_gram_counts.keys())[0]) # n is the n of the n-gram we use to estimate the probabilities

    log_sum = 0
    for sentence in corpus:

        sentence_perplexity = 1.0
        sentence = ["<s>"] * n + sentence + ["<e>"]
        sentence = tuple(sentence)

        N_i = len(sentence) - n

        for t in range(n, N_i+n):
            word = sentence[t]
            previous_n_gram = sentence[t-n:t]
            probability = estimate_probability_smoothing(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=k)
            sentence_perplexity *= probability

        if sentence_perplexity != 0:
            log_sum += math.log2(sentence_perplexity)

    perplexity = 2**(-log_sum/N)

    return perplexity

In [23]:
n_gram_counts = {('<s>', 'quick'): 1, ('the', 'quick'): 1, ('quick', 'brown'): 1, ('brown', 'fox'): 1, ('jumps', 'over'): 1, ('over', 'the'): 1, ('the', 'lazy'): 1, ('lazy', 'dog'): 1, ('dog', '<e>'): 1}
n_plus1_gram_counts = { ('<s>', '<s>', 'the', ): 1, ('<s>', 'the', 'quick'): 1, ('the', 'quick', 'brown'): 1, ('quick', 'brown', 'fox'): 1, ('jumps', 'over', 'the'): 1, ('over', 'the', 'lazy'): 1, ('the', 'lazy', 'dog'): 1, ('lazy', 'dog', '<e>'): 1}

train_corpus = [["the", "quick", "brown", "fox"], ["jumps", "over", "the", "lazy", "dog"]]
n_gram_counts = {('<s>', '<s>'): 2, ('<s>', 'the'): 1, ('<s>', 'jumps'): 1, ('the', 'quick'): 1, ('quick', 'brown'): 1, ('brown', 'fox'): 1, ('fox', '<e>'): 1, ('jumps', 'over'): 1, ('over', 'the'): 1, ('the', 'lazy'): 1, ('lazy', 'dog'): 1, ('dog', '<e>'): 1}
n_plus1_gram_counts = {('<s>', '<s>', '<s>', ): 2, ('<s>', '<s>', 'the', ): 1, ('<s>', 'the', 'quick'): 1,  ('<s>', '<s>', 'jumps', ): 1, ('<s>', 'jumps', 'over'): 1, ('the', 'quick', 'brown'): 1, ('quick', 'brown', 'fox'): 1, ('brown', 'fox', '<e>'): 1, ('jumps', 'over', 'the'): 1, ('over', 'the', 'lazy'): 1, ('the', 'lazy', 'dog'): 1, ('lazy', 'dog', '<e>'): 1}

vocabulary = ["the", "quick", "brown", "fox", "jumps", "over", "lazy", "dog", "<e>"]

test_corpus = [["the", "fox"], ["jumps"]]

# Complétez le calcul de la perplexité avec k=1
perplexity_test = calculate_perplexity_corpus(test_corpus, n_gram_counts, n_plus1_gram_counts, len(vocabulary), k=1.0)
print(f"Perplexité du corpus de test: {perplexity_test:.4f}")


Perplexité du corpus de test: 7.7089


In [24]:
# Calculez mainenant la perplexité de votre corpus de test

X_test_perplexity_bigram = calculate_perplexity_corpus(X_test, X_train_bigrams, X_train_trigrams, len(X_train_vocabulary), k=1.0)
X_test_perplexity_trigram = calculate_perplexity_corpus(X_test, X_train_trigrams, X_train_quadgrams, len(X_train_vocabulary), k=1.0)
X_test_perplexity_quadgram = calculate_perplexity_corpus(X_test, X_train_quadgrams, X_train_quintgrams, len(X_train_vocabulary), k=1.0)

print(f"Perplexité du corpus de test (bi): {X_test_perplexity_bigram:.4f}")
print(f"Perplexité du corpus de test (tri): {X_test_perplexity_trigram:.4f}")
print(f"Perplexité du corpus de test (quadri): {X_test_perplexity_quadgram:.4f}")

Perplexité du corpus de test (bi): 1175.7622
Perplexité du corpus de test (tri): 1860.5794
Perplexité du corpus de test (quadri): 2188.1338


In [25]:
X_train_perplexity_bigram = calculate_perplexity_corpus(X_train, X_train_bigrams, X_train_trigrams, len(X_train_vocabulary), k=1.0)
X_train_perplexity_trigram = calculate_perplexity_corpus(X_train, X_train_trigrams, X_train_quadgrams, len(X_train_vocabulary), k=1.0)
X_train_perplexity_quadgram = calculate_perplexity_corpus(X_train, X_train_quadgrams, X_train_quintgrams, len(X_train_vocabulary), k=1.0)

print(f"Perplexité du corpus d'entrainement (bi): {X_train_perplexity_bigram:.4f}")
print(f"Perplexité du corpus d'entrainement (tri): {X_train_perplexity_trigram:.4f}")
print(f"Perplexité du corpus d'entrainement (quadri): {X_train_perplexity_quadgram:.4f}")

Perplexité du corpus d'entrainement (bi): 827.3116
Perplexité du corpus d'entrainement (tri): 1186.9642
Perplexité du corpus d'entrainement (quadri): 1342.3060


#### 3.3.2. The expected perplexities may seem counterintuitive. Compare them to the perplexities obtained on the training set for the same models. How do you explain these results and what is your conclusion?

> Comparing the perplexities of the test and training corpora, we notice that the perplexity is much lower on the training data compared to the test data. This behavior is normal because we have already seen the n-grams on which the models are trained.
<br>
<br>
What is counterintuitive is that the perplexity of the models increases with the size N of the N-gram. We seek to minimize perplexity in order to find the best model. We would therefore expect the quad-gram model to be better than the others as its context is broader.
<br>
Our hypothesis is that we do not have enough data to properly evaluate larger N-gram models.
The size of the data should also increase with the size of the N-gram in order to estimate the probabilities of word sequences.
We are evaluating sequences of longer words, so we need more data to observe more sequences.

<a name='4'></a>
## 4. Building an auto-completion model

In this last part, you will use the n-gram models constructed in previous numbers in order to make an autocompletion model.

<a name='4.1'></a>
### 4.1 Suggestion of a word from a prefix


The first step will be to construct a function that suggests a word from the first characters entered by a user, considering a single type of n-gram.

Complete the `suggest_word` function which calculates the probabilities for all possible next words and suggests the most likely word. As an additional constraint, the suggested word must start with the prefix passed as a parameter. Use your functions from number 2. (N-gram model of words) to make your predictions.

This function takes as parameter:
- previous_n_gram: the previous n-gram, in tuple form
- n_gram_counts: A dictionary where the key is the n-gram and the value is the frequency of that n-gram.
- n_plus1_gram_counts: Another dictionary, which you will use to find the frequency of the previous n-gram plus the current word.
- vocabulary_size: the size of the vocabulary
- k: the smoothing constant
- prefix: The beginning of the word we want to predict

It returns the most probable word with the associated probability

In [26]:

def suggest_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, prefixe=""):

    n = len(list(n_gram_counts.keys())[0]) # n is the n of the n-gram we use to estimate the probabilities

    previous_n_gram = previous_tokens[-n:]
    
    suggestion = None
    max_prob = 0

    probabilities = estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=k)
    for word, prob in probabilities.items():
        if prob > max_prob and word.startswith(prefixe):
            max_prob = prob
            suggestion = word

    return suggestion, max_prob

In [27]:
# test
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = build_voc(sentences, threshold=0) # Build the vocabulary with the build_voc function that also adds the <e> token

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ["i", "have"]
tmp_suggest1 = suggest_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(f"avec les mots précédents 'i have',\n\t le mot suggéré est `{tmp_suggest1[0]}` avec la probabilité {tmp_suggest1[1]:.4f}")

print()


tmp_starts_with = 'm'
tmp_suggest2 = suggest_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0, prefixe=tmp_starts_with)
print(f"avec les mots précédents 'i have', et une suggestion qui commence par `{tmp_starts_with}`\n\t le mot suggéré est : `{tmp_suggest2[0]}` avec une probabilité de {tmp_suggest2[1]:.4f}")

avec les mots précédents 'i have',
	 le mot suggéré est `a` avec la probabilité 0.2222

avec les mots précédents 'i have', et une suggestion qui commence par `m`
	 le mot suggéré est : `mouse` avec une probabilité de 0.1111


<a name='4.2'></a>
### 4.2 Multiple suggestions

In order to suggest several words to the user, one strategy that one can use is to return a set of words suggested by several types of n-gram models.

Using the `suggest_word` function from the previous issue, complete the `get_suggestions` function which returns the suggestions from the n-gram models passed as a parameter. You will also need to remove duplicates in the suggestions if there are any, and order the list of suggestions starting with the word with the highest probability.

The get_suggestions function takes as parameters:
- previous_n_gram: the previous n-gram, in tuple form
- n_gram_counts_list: a list of n-grams in the following order [unigrams, bigrams, trigrams, quadrigrams, ...]
- vocabulary_size: the size of the vocabulary
- k: the smoothing constant (between 0 and 1)
- prefix: The beginning of the word we want to predict, "" if no prefix

In [28]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, prefixe=""):

    model_counts = len(n_gram_counts_list)

    suggestions = set()
    for i in range(model_counts - 1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]

        n = len(list(n_gram_counts.keys())[0]) # n is the n of the n-gram we use to estimate the probabilities

        if len(previous_tokens) >= n: # Only evalute n-grams that don't exceed the length of the sentence. MWe are look at n-grams that come after the previous tokens.
            suggestion = suggest_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=k, prefixe=prefixe)
            suggestions.add(suggestion)

    # Sort suggestions
    suggestions = sorted(suggestions, key=lambda x: x[1], reverse=True)

    # Return only the words
    suggestions = [tup[0] for tup in suggestions]

    # keep only unique values
    suggestions = list(set(suggestions))

    return suggestions

In [29]:
# test
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
quadgram_counts = count_n_grams(sentences, 4)
qintgram_counts = count_n_grams(sentences, 5)

n_gram_counts_list = [unigram_counts, bigram_counts, trigram_counts, quadgram_counts, qintgram_counts]
previous_tokens = ["i", "have"]
tmp_suggest3 = get_suggestions(previous_tokens, n_gram_counts_list, unique_words, k=1.0)

print(f"Etant donné les mots i have, je suggère :")
display(tmp_suggest3)

Etant donné les mots i have, je suggère :


['a']

<a name='4.3'></a>
### 4.3 Autocompletion

Now it's time to combine your functions to create the autocomplete template. Using the training dataset, calculate the frequency of n-grams ranging from 1 to 5 and use the *get_suggestions* function to suggest words. You will need to be able to always suggest words starting from the last word entered by the user.

Complete the *update_suggestions* function:
- the current_text variable contains all the text entered by the user
- the top_suggestions variable contains the suggestions that will be offered

You will need to change the contents of the top_suggestions variable so that it contains the n-gram suggestions.

In [30]:
X_train_n_gram_counts_list = [X_train_unigrams, X_train_bigrams, X_train_trigrams, X_train_quadgrams, X_train_quintgrams]

In [31]:
import ipywidgets as widgets
from IPython.display import display


text_input = widgets.Text(placeholder="Entrez votre text ici...")

suggestions_label = widgets.Label(value="Suggestions: ")

def update_suggestions(change):
     texte_actuel = change["new"]

     if texte_actuel != "":
         # Tokenize the text
         text_data = preprocess(texte_actuel)

         # Replace out-of-vocabulary words
         tokenized_text = replace_oov(text_data, X_train_vocabulary, False)

         # Get suggestions
         top_suggestions = get_suggestions(tokenized_text[0], X_train_n_gram_counts_list, X_train_vocabulary, k=1.0, prefixe="")

         suggestions_label.value = "Suggestions: " + ", ".join(top_suggestions)

text_input.observe(update_suggestions, names="value")

display(text_input)
display(suggestions_label)

Text(value='', placeholder='Entrez votre text ici...')

Label(value='Suggestions: ')

<a name='5'></a>
## 5. Sentence generation model

In this part you will build a sentence generation model using n-grams.

#### As part of a sentence generation model, indicate why the word suggestion strategy in 4. cannot work?

>In question 4, we return the most probable word according to the n-gram. This is problematic in the context of sentence generation because we are starting from a non-existent context, so the same word will be judged most likely to start the sentence. So the first word will always be the same and the following words won't have much variety because they will all depend on the same context. An added stochastic effect could prevent this problem.

<a name='5.1'></a>

### 5.1 Stochastic generation of words

Recode the suggest_word function to use a stochastic suggestion. In other words, instead of returning the most probable word, you will have to generate the next word according to its probability.

For example if the word 'like' has probability 0.25 of being generated, then it will be returned 25% of the time.

In [32]:
import random

def suggest_word_with_probs(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    
    n = len(list(n_gram_counts.keys())[0]) # n is the n of the n-gram we use to estimate the probabilities

    previous_n_gram = previous_tokens[-n:]

    probabilities = estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=k)
    words = list(probabilities.keys())
    probs = list(probabilities.values())
    
    chosen_word = random.choices(words, weights=probs, k=1)[0]
    return chosen_word

<a name='5.2'></a>
### 5.2 Sentence generations

#### 5.2.1. Stochastic generation
Now complete the `generate_sentence` function which generates an n_words long sentence by calling your new function `suggest_words_with_probs`. Generation should stop if the model generates an end-of-sentence token.

Don't forget to initialize the sentences to be generated with the right number of sentence start tokens (`<s>`). For example, if it is a bigram model, you will have to initialize the sentence at [`<s>`]. If it is a trigram model, you will have to initialize the sentence at [`<s>`, `<s>`]. You can find the context size using the following expression `len(next(iter(n_gram_counts)))`.

Then, you will need to pass to the `suggest_word` function the last `n` words generated where `n` corresponds to the size of the context.
Finally, you will have to stop the generation if the generated token is the end token (`<e>`)

In [33]:
def generate_sentence(n_words, n_gram_counts, n_plus1_gram_counts, vocabulary, k=0.0001):
    n = len(next(iter(n_gram_counts)))
    
    sentence = ['<s>'] * n
    
    for _ in range(n_words):
        next_word = suggest_word_with_probs(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary, k=k)

        if next_word == '<e>':
            break
            
        sentence.append(next_word)
        
    return sentence[n:]


#### 5.2.2. Test on n-grams
Then test your function with trigrams and 5-grams.

In [34]:
trigram_sentence = generate_sentence(10, X_train_trigrams, X_train_quadgrams, X_train_vocabulary)
print(f"Phrase générée pour trigramme: {trigram_sentence}")

Phrase générée pour trigramme: ['we', 'will', 'cancel', 'revived', 'rogue', 'tougher', 'hospital', 'instance', 'intelligently', 'radicalism', 'dudes', 'affect', 'mayors', 'drivers', 'excitement', 'trees', 'flag', '500', 'riley', 'jetfighters']


In [42]:
X_train_sixgrams = count_n_grams(X_train, 6)
quintgram_sentence = generate_sentence(10, X_train_quintgrams, X_train_sixgrams, X_train_vocabulary)
print(f"Phrase générée pour 5-gramme: {quintgram_sentence}")

Phrase générée pour 5-gramme: ['ive', 'outlined', 'a', 'plan', 'to', 'provide', 'every', 'disadvantaged', 'child', 'franklin', 'break', 'kind', 'golfer', 'farms', 'renegotiate', 'authorize', 'truck', 'vanishing', 'post', 'cheap']


#### 5.2.3. With k=1.0, what happens to the generated sentences and what is the main reason? What can you do to improve the situation?

>The generated sentences are no longer coherent.
The smoothing is too strong and the probabilities are therefore falsely evaluated.
<br>
Words that should have low probabilities are rated higher and are suggested by the algorithm.
<br>
<br>
It is therefore necessary to adjust the parameter K so that the model suggests coherent sentences

#### 5.2.4. What are the problems if the constant k has a value too small, see 0?

>With k=0, we find ourselves in a sort of over-fitting situation.
<br>
The probabilities of n-grams missing from the training data drop to 0.
<br>
Thus, only the n-grams seen in training are considered during the random selection.

<a name='5.3'></a>
### 5.3. Improved stochastic word generation

#### 5.3.1. Stochastic improvement

As you may have observed, stochastic generation, although effective at generating different sentences, tends not to generate consistently coherent sentences. Propose an improvement to the `suggest_word` method that you will implement in the `suggest_word_new` method to generate more coherent sentences.

##### a) Describe your method in the following cell

>suggest_word_with_probs gives a probability to out-of-context words with a very low chance. We can remove a degree of stochasticity by randomly drawing from the x most probable words according to a given context.

##### b) Implement the proposed method

In [36]:

def suggest_word_new(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    x = 20 # we choose the top n words to keep and choose randomly between those
    n = len(list(n_gram_counts.keys())[0]) # n is the n of the n-gram we use to estimate the probabilities

    previous_n_gram = previous_tokens[-n:]

    probabilities = estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=k)
    sorted_probs = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)[:x]

    words, probs = zip(*sorted_probs)
    chosen_word = random.choices(words, weights=probs)[0]

    return chosen_word

#### 5.3.2. Improved generation
Now recode the `generate_sentence_new` function to call your new `suggest_word_new` method.

In [37]:
def generate_sentence_new(n_words, n_gram_counts, n_plus1_gram_counts, vocabulary, k=0.001):
    n = len(next(iter(n_gram_counts)))
    
    sentence = ['<s>'] * n
        
    for _ in range(n_words):
        next_word = suggest_word_new(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary, k=k)

        if next_word == '<e>':
            break

        sentence.append(next_word)

    return sentence[n:]

#### 5.3.3. Test on n-grams
Then test your function with 3-grams and 5-grams and validate that the sentences are more coherent.

In [38]:
trigram_sentence = generate_sentence_new(20, X_train_trigrams, X_train_quadgrams, X_train_vocabulary)
print(f"Phrase générée pour le trigramme: {trigram_sentence}")

Phrase générée pour le trigramme: ['the', 'great', 'state', 'of', 'pennsylvania']


In [39]:
quintgram_sentence = generate_sentence_new(20, X_train_quintgrams, X_train_sixgrams, X_train_vocabulary)
print(f"Phrase générée pour le quintgramme: {quintgram_sentence}")

Phrase générée pour le quintgramme: ['were', 'gon', 'na', 'build', 'it', 'up']
